# Considerações
**Altura de cada andar: 3m (exceto subsolo e área serviço)**

**Considera o valor "4" uma vez por considerar apenas a área de serviço**

**Então, erro máximo = 4 + 3 x (8-1) = 25**

**A faixa de tolerância é entre 0 e 10 cm, tendo o valor de pertinência igual a 1. Entre 10 cm e 20 cm, terá uma pertinência.**

<img src="Assets/prédio.png" style="height:500px;width:400px">

# Integrantes

**Matheus Camara Carvalho**

**Igor da Silva Villamarim**

**Gabriel de Souza Gemelle Leal**

### 1. Instalando bibliotecas necessárias

In [ ]:
%pip install paho-mqtt scikit-fuzzy python-dotenv

### 2. Importação de bibliotecas

In [ ]:
import os
from dotenv import load_dotenv
import numpy as np
import skfuzzy as fuzzy
import time
import paho.mqtt.client as paho
import itertools
from paho import mqtt
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import messagebox

### 3. Iniciando variáveis mqtt e o HiveMQ

In [ ]:
# Especificar o caminho do arquivo .env
dotenv_path = 'Variables.env'
if not os.path.exists(dotenv_path):
    raise FileNotFoundError(f"Arquivo {dotenv_path} não encontrado.")

load_dotenv(dotenv_path)

# Obtém as variáveis de ambiente - Segurança adicional para evitar o vazamento das credenciais em texto claro
mqtt_username = os.getenv('MQTT_USERNAME')
mqtt_password = os.getenv('MQTT_PASSWORD')
mqtt_url = os.getenv('MQTT_URL')
mqtt_port = os.getenv('MQTT_PORT')

# Verificar se todas as variáveis de ambiente foram carregadas corretamente
if not all([mqtt_username, mqtt_password, mqtt_url, mqtt_port]):
    raise ValueError("Uma ou mais variáveis de ambiente não foram carregadas corretamente.")

mqtt_port = int(mqtt_port)  # Converter para inteiro

# Definindo callbacks para diferentes eventos para ver se funciona, imprimir a mensagem etc.
def on_connect(client, userdata, flags, rc, properties=None):
    if rc == 0:
        print("Conexão estabelecida com sucesso!")
    else:
        print("CONNACK received with code %s." % rc)

# Com este callback você pode ver se sua publicação foi bem-sucedida
def on_publish(client, userdata, mid, properties=None):
    print(f"Publicação realizada com sucesso! (mid={mid})")

# Imprimir qual tópico foi inscrito
def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

# Imprimir mensagem, útil para verificar se foi bem-sucedido
def on_message(client, userdata, msg):
    print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))

client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)
client.on_connect = on_connect  

# Habilita o TLS para uma conexão segura (diferencial) 
client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)

# Configurar nome de usuário e senha
client.username_pw_set(mqtt_username, mqtt_password)
# Conectar ao HiveMQ Cloud na porta especificada
try:
    client.connect(mqtt_url, mqtt_port)
except Exception as e:
    print(f"Erro ao conectar: {e}")
    
client.publish("encyclopedia/temperature", payload="hot", qos=1)

# Configurar callbacks
client.on_subscribe = on_subscribe
client.on_message = on_message
client.on_publish = on_publish



### 4. Variáveis de Erro

In [ ]:
# Erro máximo = 25
# Os valores estão referenciados em cm, valores decimais não são suficientes

# Variável erro de entrada do sistema de controle
Erro = ctrl.Antecedent(universe=np.arange(0, 25.01, 0.01), label='Erro')

#trapmf = trapezoidal
#trimf = triangular
Erro['ZE'] = fuzzy.trapmf(Erro.universe, [0, 0, 0.1, 0.2])
Erro['R1'] = fuzzy.trimf(Erro.universe, [0.15, 0.2, 9])
Erro['R2'] = fuzzy.trimf(Erro.universe, [0.2, 9, 15])
Erro['R3'] = fuzzy.trimf(Erro.universe, [9, 15, 21])
Erro['R4'] = fuzzy.trimf(Erro.universe, [15, 21, 25])

Erro.view()
plt.axhline(y=0.5, color='r', linestyle='--')
[plt.gca().lines[i].set_linewidth(2) for i in range(len(plt.gca().lines))]

#A linha em 0.5 no eixo horizontal serve apenas para mostrar que as funções cruzam nesse ponto
figura = plt.gcf(); axes = figura.gca(); figura.set_size_inches(6, 2) 
axes.set_xlabel(xlabel= 'Erro [m]'); axes.set_ylabel(ylabel='Pertinência $\mu_{E}$')
plt.legend(loc='upper right'); plt.savefig('Output/Erro.png')
plt.xticks([])


### 5. Variáveis de ∆Erro

In [ ]:
# Delta Erro máximo = +-50 (Sempre o dobro do erro)
# Como a tolerância foi de 10cm, o delta erro de tolerância também será o dobro (ZE)

# Variável erro de entrada do sistema de controle
dErro = ctrl.Antecedent(universe=np.arange(-50, 50.1, 0.1), label='∆Erro')

dErro['MN'] = fuzzy.trapmf(dErro.universe, [-50, -50, -24, -12])
dErro['PN'] = fuzzy.trimf(dErro.universe, [-24, -0.2, 0])
dErro['ZE'] = fuzzy.trimf(dErro.universe, [-0.2, 0, 0.2])
dErro['PP'] = fuzzy.trimf(dErro.universe, [0, 0.2, 24])
dErro['MP'] = fuzzy.trapmf(dErro.universe, [12, 24, 50, 50])

dErro.view()
[plt.gca().lines[i].set_linewidth(2) for i in range(len(plt.gca().lines))]
figura = plt.gcf(); axes = figura.gca(); figura.set_size_inches(6, 2) 
axes.set_xlabel(xlabel= '$\Delta_{Erro} [m]$'); axes.set_ylabel(ylabel='Pertinência $\mu_{\Delta_{E}}$')
plt.legend(loc='upper right'); plt.savefig('Output/DeltaErro.png')
plt.xticks([])

### 6. Variáveis da Potência do Motor

In [ ]:
PotenciaMotor = ctrl.Consequent(universe=np.arange(0, 90.5, 0.5), label='PotenciaMotor')

PotenciaMotor['I'] = fuzzy.trimf(PotenciaMotor.universe, [0, 0, 31.5])
PotenciaMotor['B'] = fuzzy.trimf(PotenciaMotor.universe, [15, 31.5, 45])
PotenciaMotor['M'] = fuzzy.trimf(PotenciaMotor.universe, [31.5, 45.0, 90])
PotenciaMotor['A'] = fuzzy.trimf(PotenciaMotor.universe, [45.0, 90.0, 90])

PotenciaMotor.view()
[plt.gca().lines[i].set_linewidth(2) for i in range(len(plt.gca().lines))]

figura = plt.gcf(); axes = figura.gca(); figura.set_size_inches(6, 2) 
axes.set_xlabel(xlabel= '$P_{Motor}$ [%]'); axes.set_xlim([0, 100])
axes.set_ylabel(ylabel='Pertinência $\mu_{\Delta_{E}}$')
plt.legend(loc='upper right'); plt.savefig('Output/Aceleração.png')
plt.xticks([])

### 7. Base de Regras

In [ ]:
import skfuzzy.control as ctrl

R1 = ctrl.Rule(Erro['ZE'] & dErro['MN'], PotenciaMotor['I'])
R2 = ctrl.Rule(Erro['R1'] & dErro['MN'], PotenciaMotor['I'])
R3 = ctrl.Rule(Erro['R2'] & dErro['MN'], PotenciaMotor['B'])
R4 = ctrl.Rule(Erro['R3'] & dErro['MN'], PotenciaMotor['M'])
R5 = ctrl.Rule(Erro['R4'] & dErro['MN'], PotenciaMotor['A'])

R6 = ctrl.Rule(Erro['ZE'] & dErro['PN'], PotenciaMotor['I'])
R7 = ctrl.Rule(Erro['R1'] & dErro['PN'], PotenciaMotor['B'])
R8 = ctrl.Rule(Erro['R2'] & dErro['PN'], PotenciaMotor['M'])
R9 = ctrl.Rule(Erro['R3'] & dErro['PN'], PotenciaMotor['A'])
R10 = ctrl.Rule(Erro['R4'] & dErro['PN'], PotenciaMotor['A'])

R11 = ctrl.Rule(Erro['ZE'] & dErro['ZE'], PotenciaMotor['I'])
R12 = ctrl.Rule(Erro['R1'] & dErro['ZE'], PotenciaMotor['B'])
R13 = ctrl.Rule(Erro['R2'] & dErro['ZE'], PotenciaMotor['M'])
R14 = ctrl.Rule(Erro['R3'] & dErro['ZE'], PotenciaMotor['A'])
R15 = ctrl.Rule(Erro['R4'] & dErro['ZE'], PotenciaMotor['A'])

R16 = ctrl.Rule(Erro['ZE'] & dErro['PP'], PotenciaMotor['B'])
R17 = ctrl.Rule(Erro['R1'] & dErro['PP'], PotenciaMotor['M'])
R18 = ctrl.Rule(Erro['R2'] & dErro['PP'], PotenciaMotor['A'])
R19 = ctrl.Rule(Erro['R3'] & dErro['PP'], PotenciaMotor['A'])
R20 = ctrl.Rule(Erro['R4'] & dErro['PP'], PotenciaMotor['A'])

R21 = ctrl.Rule(Erro['ZE'] & dErro['MP'], PotenciaMotor['M'])
R22 = ctrl.Rule(Erro['R1'] & dErro['MP'], PotenciaMotor['A'])
R23 = ctrl.Rule(Erro['R2'] & dErro['MP'], PotenciaMotor['A'])
R24 = ctrl.Rule(Erro['R3'] & dErro['MP'], PotenciaMotor['A'])
R25 = ctrl.Rule(Erro['R4'] & dErro['MP'], PotenciaMotor['A'])

# Criação do sistema de controle
Elevador_ctrl = ctrl.ControlSystem([R1, R2, R3, R4, R5,
                                     R6, R7, R8, R9, R10,
                                     R11, R12, R13, R14, R15,
                                     R16, R17, R18, R19, R20,
                                     R21, R22, R23, R24, R25])

controlador = ctrl.ControlSystemSimulation(Elevador_ctrl)

### 8. Print da Base de Regras

In [ ]:
QuantidadeRegras = len(Erro.terms)*len(dErro.terms)
BaseRegras = [globals()[f'R{i}'] for i in range(1, QuantidadeRegras + 1)]
BaseRegras

### 9. Simulação de sistema de controle fuzzy

In [ ]:
ControleVelocidade = ctrl.ControlSystemSimulation(ctrl.ControlSystem(BaseRegras))

In [ ]:
AndarAtual = 8
AndarDeslocado = 11

posicao, posicaoAtual = [[AndarAtual], AndarAtual]
ErroAltura = AndarDeslocado - posicaoAtual

for t in np.arange(0.1, 3, 0.1):
    PotenciaMotor = t*0.315/3
    posicaoAtual = posicaoAtual*0.996*(1 if ErroAltura > 0 else -1) + PotenciaMotor * 0.00951
    if t == round(t, 0): posicao = np.append(posicao, posicaoAtual)

ErroAnterior = AndarDeslocado - posicaoAtual

for _ in np.arange(3, 400, 1):
    ErroAltura = AndarDeslocado - posicaoAtual
    ErroAtual = abs(ErroAltura)
    ControleVelocidade.input[Erro.label] = ErroAtual
    dErroAtual = ErroAnterior - ErroAtual
    ControleVelocidade.input[dErro.label] = dErroAtual
    ControleVelocidade.compute()
    PotenciaMotor = ControleVelocidade.output['PotenciaMotor']
    posicaoAtual = abs(posicaoAtual*0.996*(1 if ErroAltura > 0 else -1) + PotenciaMotor*0.00951)

    print(f'{ErroAtual:.2f} {dErroAtual:.2f} -> {PotenciaMotor:.2f} -> {posicaoAtual:.2f}')

    posicao = np.append(posicao, posicaoAtual)
    ErroAnterior = ErroAtual

plt.plot(np.arange(1, 401, 1), posicao)
plt.savefig('Output/Simulacao.png')

### 11. Entrada e Saída de dados com entrada estática

### 11. Entrada e Saída de dados com entrada dinâmica

In [ ]:
from IPython.display import clear_output


# Função para simular o movimento do elevador e publicar no MQTT
def simular_movimento(altura_andar_atual, altura_andar_destino):
    global client
    posicao, posicaoAtual = [[altura_andar_atual], altura_andar_atual]
    ErroAltura = altura_andar_destino - posicaoAtual

    for t in np.arange(0.1, 3, 0.1):
        PotenciaMotor = t * 0.315 / 3
        posicaoAtual = posicaoAtual * 0.996 * (1 if ErroAltura > 0 else -1) + PotenciaMotor * 0.00951
        if t == round(t, 0): posicao = np.append(posicao, posicaoAtual)

    ErroAnterior = altura_andar_destino - posicaoAtual

    for _ in np.arange(3, 400, 1):
        ErroAltura = altura_andar_destino - posicaoAtual
        ErroAtual = abs(ErroAltura)
        ControleVelocidade.input[Erro.label] = ErroAtual
        dErroAtual = ErroAnterior - ErroAtual
        ControleVelocidade.input[dErro.label] = dErroAtual
        ControleVelocidade.compute()
        PotenciaMotor = ControleVelocidade.output['PotenciaMotor']
        posicaoAtual = abs(posicaoAtual * 0.996 * (1 if ErroAltura > 0 else -1) + PotenciaMotor * 0.00951)
        print(f'{ErroAtual:.2f} {dErroAtual:.2f} -> {PotenciaMotor:.2f} -> {posicaoAtual:.2f}')
        
        # Publicar no MQTT 
        client.publish("elevador/ErroAtual", payload=ErroAtual)
        client.publish("elevador/dErroAtual", payload=dErroAtual)
        client.publish("elevador/PotenciaMotor", payload=PotenciaMotor)
        
        posicao = np.append(posicao, posicaoAtual)
        ErroAnterior = ErroAtual

# Função para processar a escolha do usuário
def processar_escolha(opcao):
    global andar_atual, andar_destino, distancia_andar, mapeamento_andares, alturas_andares

    if opcao.startswith('Atual_'):
        andar_escolhido = opcao.split('_')[1]
        if andar_atual == andar_escolhido:
            messagebox.showwarning("Aviso", f"Você já está no andar {mapeamento_andares[andar_atual][1]}! Escolha outro destino.")
        else:
            andar_atual = andar_escolhido
            altura_andar_atual = alturas_andares[mapeamento_andares[andar_atual][1]]
            messagebox.showinfo("Informação", f"Andar atual selecionado: {mapeamento_andares[andar_atual][1]} ({altura_andar_atual} metros)")
            # Habilitar os botões de destino após escolher o andar atual
            habilitar_botoes_destino(True)
    elif opcao.startswith('Destino_'):
        andar_escolhido = opcao.split('_')[1]
        if andar_atual is None:
            messagebox.showwarning("Aviso", "Por favor, escolha primeiro o andar atual.")
        elif andar_atual == andar_escolhido:
            messagebox.showwarning("Aviso", "Você já está no andar de destino! Escolha outro destino.")
        else:
            andar_destino = andar_escolhido
            altura_andar_atual = alturas_andares[mapeamento_andares[andar_atual][1]]
            altura_andar_destino = alturas_andares[mapeamento_andares[andar_destino][1]]
            deslocamento = altura_andar_destino - altura_andar_atual
            direcao = "subindo" if deslocamento >= 0 else "descendo"
            messagebox.showinfo("Informação", f"Andar de destino selecionado: {mapeamento_andares[andar_destino][1]} (Deslocamento em metros: {abs(deslocamento)} {direcao})")

            # Limpar a saída do Jupyter Notebook
            clear_output(wait=False)

            # Chama a função para simular o movimento e publicar no MQTT
            simular_movimento(altura_andar_atual, altura_andar_destino)



# Função para habilitar ou desabilitar os botões de destino
def habilitar_botoes_destino(habilitar):
    global frame_destino

    for widget in frame_destino.winfo_children():
        widget.configure(state=tk.NORMAL if habilitar else tk.DISABLED)

# Criando a janela principal
root = tk.Tk()
root.title("Elevador")

# Definindo variáveis globais para armazenar a escolha do usuário
andar_atual = None
andar_destino = None
distancia_andar = 3  # Distância entre cada andar em metros

# Dicionário para mapear letras para números de andares e suas alturas correspondentes
mapeamento_andares = {
    'T': (4, 0),    # Altura, andar correspondente
    '1': (8, 1),
    '2': (11, 2),
    '3': (14, 3),
    '4': (17, 4),
    '5': (20, 5),
    '6': (23, 6),
    '7': (26, 7),
    '8': (29, 8)
}

# Ajustando alturas_andares para acessar a altura correta de cada andar
alturas_andares = {andar: altura for altura, andar in mapeamento_andares.values()}

# Frame para os botões do andar atual
frame_atual = tk.LabelFrame(root, text="Andar Atual")
frame_atual.pack(padx=10, pady=10, side=tk.LEFT)

# Botões do andar atual
opcoes_atual = ["T", "1", "2", "3", "4", "5", "6", "7", "8"]
for opcao in opcoes_atual:
    botao = tk.Button(frame_atual, text=f"{opcao}", command=lambda opcao=opcao: processar_escolha(f"Atual_{opcao}"))
    botao.pack(fill=tk.X, padx=10, pady=5)

# Frame para os botões do andar de destino
frame_destino = tk.LabelFrame(root, text="Andar de Destino")
frame_destino.pack(padx=10, pady=10, side=tk.RIGHT)

# Botões do andar de destino
opcoes_destino = ["T", "1", "2", "3", "4", "5", "6", "7", "8"]
for opcao in opcoes_destino:
    botao = tk.Button(frame_destino, text=f"{opcao}", command=lambda opcao=opcao: processar_escolha(f"Destino_{opcao}"))
    botao.pack(fill=tk.X, padx=10, pady=5)
    # Inicialmente desabilitar os botões de destino
    botao.configure(state=tk.DISABLED)

# Rodando o loop principal do tkinter
root.mainloop()

